## Transformer

Transformers only reads in the following formats:

1. txt
2. csv
3. tsv
4. jsonl
5. json
6. xml

[Source](https://huggingface.co/docs/datasets/dataset_script.html)

If you want to read in pdf files, you would have to convert it to any of the format above.

for our notebook we would be converting thekkkkkkkkknzkjx000000m t

In [ ]:
!pip install transformers
!pip install datasets
!pip install pdfminer
!pip install rouge_score
!pip install rouge

In [ ]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from datasets import load_metric



#from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
metric = load_metric('rouge')

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
output_string = StringIO()
with open("summary_one.pdf" , "rb") as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device =  TextConverter(rsrcmgr, output_string, laparams= LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
        #print('*' * 100)
        
print(output_string.getvalue())

There were certain underlying conditions that enabled department stores to
grow up when they did. From the start, they all catered for middle-class
customers and set out to convey to them an air of luxury and solid comfort. Of
necessity, they all arose in central positions where large numbers of people
could reach them easily by means of public transport. Physically, they grew up
in an era of big technical developments in building so that they could a ord
multi-storey palaces and could have enormous plate-glass windows for
display, lighting and novelties like lifts.
Above all, the department stores rose with the rise of Victorian white-collar
workers, the small-scale businessmen and professionals whose womenfolk
had money to spare for a few luxuries and were gradually switching the
emphasis of their housekeeping expenditure from food to other items.

Most of these stores drew enough customers to ﬁll their huge shops by o ering
two new things. One was the new manufactures, particularly 

In [ ]:
text = output_string.getvalue()
text = text[:-4]
dataset = text.replace('\n', ' ').replace('\\', '').replace('\n\n', '').replace('\x0c','').replace('\x00', 'ff')
dataset

'There were certain underlying conditions that enabled department stores to grow up when they did. From the start, they all catered for middle-class customers and set out to convey to them an air of luxury and solid comfort. Of necessity, they all arose in central positions where large numbers of people could reach them easily by means of public transport. Physically, they grew up in an era of big technical developments in building so that they could afford multi-storey palaces and could have enormous plate-glass windows for display, lighting and novelties like lifts. Above all, the department stores rose with the rise of Victorian white-collar workers, the small-scale businessmen and professionals whose womenfolk had money to spare for a few luxuries and were gradually switching the emphasis of their housekeeping expenditure from food to other items.  Most of these stores drew enough customers to ﬁll their huge shops by offering two new things. One was the new manufactures, particular

In [35]:

n =512
bank = [dataset[i: i + len(dataset)//3 + 2] for i in range(0, len(dataset), len(dataset)//3 +2)]
bank[0]

'There were certain underlying conditions that enabled department stores to grow up when they did. From the start, they all catered for middle-class customers and set out to convey to them an air of luxury and solid comfort. Of necessity, they all arose in central positions where large numbers of people could reach them easily by means of public transport. Physically, they grew up in an era of big technical developments in building so that they could afford multi-storey palaces and could have enormous plate-glass windows for display, lighting and novelties like lifts. Above all, the department stores rose with the rise of Victorian white-collar workers, the small-scale businessmen and professionals whose womenfolk had money to spare for a few luxuries and were gradually switching the emphasis of their housekeeping expenditure from food to other '

In [36]:
bank[1]

'items.  Most of these stores drew enough customers to ﬁll their huge shops by offering two new things. One was the new manufactures, particularly clothing, goods and accessories, household furnishings and equipment of all kinds that were coming out of the factories in increasing quantities. The specialist shops stocked these too, of course, but the department stores always made it a point to be the ﬁrst in the ﬁeld if they could with novelty of any kind. And the other special thing they offered the middle-class shoppers, many of whom were newly rich and a little experienced in luxury shopping, was a lavish display and wide choice of these goods.  The department stores, however, introduced into a respectable class trade the vulgar practice of openly marking or ticketing goods with their prices – a practice that had not even yet penetrated shops '

In [37]:
bank[2]

'that could claim that they were really exclusive. But the department stores as a rule made a virtue not only of displaying their wares as openly as they could but also of boldly pricing them for all to see. Their large-scale purchases enabled them to sell cheaply and they were not ashamed in the early days to make price one of their selling points. ‘Store price’ was a by-word for cheapness.  The lines they concentrated on were fashion goods, things that shoppers were prepared to travel long distances for and to take some time and trouble in choosing. The department stores were at least partly responsible for the way the middle classes gradually became fashion conscious, and helped to mould their tastes. They were the ﬁrst preachers of the modern creed that goods  ought to be replaced when they are outdated rather than when they are outworn'

In [39]:

#n 512=
#bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

#bank = [dataset[i: i + len(dataset)//2+1] for i in range(0, len(dataset), len(dataset)//2 +1)]

bank = [dataset[i: i + len(dataset)//3 + 2] for i in range(0, len(dataset), len(dataset)//3 +2)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=0.0, num_beams=5, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)



 department stores rose with the rise of Victorian white-collar workers. small-scale businessmen and professionals whose womenfolk had money to spare for luxuries were gradually switching emphasis of housekeeping expenditure from food to other expenditures. department stores made it a point to be the first in the field if they could with novelty of any kind. other special thing they offered the middle-class shoppers was lavish display and wide choice of these goods. department stores were at least partly responsible for the way the middle classes gradually became fashion conscious. they were the first preachers of the modern creed that goods ought to be replaced when they are outdated rather than outworn.


In [ ]:
#lenght penalty < 0 batch size of 3

#department stores rose with the rise of Victorian white-collar workers. 
#small-scale businessmen and professionals whose womenfolk had money to spare for luxuries were gradually switching emphasis of housekeeping expenditure from food to other expenditures. 
#department stores made it a point to be the first in the field if they could with novelty of any kind. 
#other special thing they offered the middle-class shoppers was lavish display and wide choice of these goods. 
#department stores were at least partly responsible for the way the middle classes gradually became fashion conscious. 
#they were the first preachers of the modern creed that goods ought to be replaced when they are outdated rather than outworn.


In [43]:
metric.compute(predictions = [result], references = [dataset])

{'rouge1': AggregateScore(low=Score(precision=0.9923076923076923, recall=0.29930394431554525, fmeasure=0.45989304812834225), mid=Score(precision=0.9923076923076923, recall=0.29930394431554525, fmeasure=0.45989304812834225), high=Score(precision=0.9923076923076923, recall=0.29930394431554525, fmeasure=0.45989304812834225)),
 'rouge2': AggregateScore(low=Score(precision=0.9069767441860465, recall=0.27209302325581397, fmeasure=0.4186046511627907), mid=Score(precision=0.9069767441860465, recall=0.27209302325581397, fmeasure=0.4186046511627907), high=Score(precision=0.9069767441860465, recall=0.27209302325581397, fmeasure=0.4186046511627907)),
 'rougeL': AggregateScore(low=Score(precision=0.8307692307692308, recall=0.2505800464037123, fmeasure=0.3850267379679144), mid=Score(precision=0.8307692307692308, recall=0.2505800464037123, fmeasure=0.3850267379679144), high=Score(precision=0.8307692307692308, recall=0.2505800464037123, fmeasure=0.3850267379679144)),
 'rougeLsum': AggregateScore(low=S

In [ ]:
#lenght penalty < 0 batch size of 2

#the department stores rose with the rise of Victorian white-collar workers. they catered for middle-class customers and set out to convey to them an air of luxury and solid comfort. 
#department stores introduced vulgar practice of openly marking or ticketing goods with their prices into class trade. 
#large-scale purchases enabled them to sell cheaply and they were not ashamed in the early days to make price one of their selling points.

In [ ]:
#lenght penalty > 0 batch size of 2

#the department stores rose with the rise of Victorian white-collar workers. they catered for middle-class customers and set out to convey to them an air of luxury and solid comfort, 
#says robert mcginnon. he says they grew up in central positions where large numbers of people could reach them easily by public transport. 
#department stores introduced vulgar practice of openly marking or ticketing goods with their prices into class trade. 
#large-scale purchases enabled them to sell cheaply and they were not ashamed in the early days to make price one of their selling points. 
#‘Store price’ was a by-word for cheapness. the lines they concentrated on were fashion goods, 
#things that shoppers were prepared to travel long distances for and to take some time in choosing.

In [ ]:
metric.compute(predictions = [result], references = [dataset])

{'rouge1': AggregateScore(low=Score(precision=0.9615384615384616, recall=0.2900232018561485, fmeasure=0.4456327985739751), mid=Score(precision=0.9615384615384616, recall=0.2900232018561485, fmeasure=0.4456327985739751), high=Score(precision=0.9615384615384616, recall=0.2900232018561485, fmeasure=0.4456327985739751)),
 'rouge2': AggregateScore(low=Score(precision=0.875968992248062, recall=0.2627906976744186, fmeasure=0.40429338103756707), mid=Score(precision=0.875968992248062, recall=0.2627906976744186, fmeasure=0.40429338103756707), high=Score(precision=0.875968992248062, recall=0.2627906976744186, fmeasure=0.40429338103756707)),
 'rougeL': AggregateScore(low=Score(precision=0.8538461538461538, recall=0.25754060324825984, fmeasure=0.3957219251336898), mid=Score(precision=0.8538461538461538, recall=0.25754060324825984, fmeasure=0.3957219251336898), high=Score(precision=0.8538461538461538, recall=0.25754060324825984, fmeasure=0.3957219251336898)),
 'rougeLsum': AggregateScore(low=Score(p

In [42]:

# n =512
# bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

#bank = [dataset[i: i + len(dataset)//2+1] for i in range(0, len(dataset), len(dataset)//2 +1)]

bank = [dataset[i: i + len(dataset)//3 + 2] for i in range(0, len(dataset), len(dataset)//3 +2)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=0.0, num_beams=4, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)


 department stores rose with the rise of white-collar workers and small-scale businessmen and professionals whose womenfolk had money to spare for few luxuries. they were gradually switching the emphasis of their housekeeping expenditure from food to other expenditures. the department stores introduced into a respectable class trade the vulgar practice of openly marking or ticketing goods with their prices. one was the new manufactures, particularly clothing, goods and accessories, household furnishings and equipment of all kinds that were coming out of the factories in increasing quantities. department stores made a virtue not only of displaying their wares as openly as they could but also of boldly pricing them for all to see. they were not ashamed in the early days to make price one of their selling points.


In [ ]:
#department stores rose with the rise of white-collar workers and small-scale businessmen and professionals whose womenfolk had money to spare for luxuries. 
#one was the new manufactures, particularly clothing, goods and accessories, household furnishings and equipment that were coming out of the factories in increasing quantities. 
#the specialist shops stocked these too, of course, but they always made it a point to be the first in the field. 
#the department stores introduced into a respectable class trade the vulgar practice of openly marking or ticketing goods with their prices. 
#but they introduced the'store price' store was at least partly responsible for the way they gradually became fashion conscious and helped to mould their tastes.

In [ ]:
metric.compute(predictions = [result], references = [dataset])

{'rouge1': AggregateScore(low=Score(precision=0.9712230215827338, recall=0.31322505800464034, fmeasure=0.47368421052631576), mid=Score(precision=0.9712230215827338, recall=0.31322505800464034, fmeasure=0.47368421052631576), high=Score(precision=0.9712230215827338, recall=0.31322505800464034, fmeasure=0.47368421052631576)),
 'rouge2': AggregateScore(low=Score(precision=0.8043478260869565, recall=0.25813953488372093, fmeasure=0.3908450704225352), mid=Score(precision=0.8043478260869565, recall=0.25813953488372093, fmeasure=0.3908450704225352), high=Score(precision=0.8043478260869565, recall=0.25813953488372093, fmeasure=0.3908450704225352)),
 'rougeL': AggregateScore(low=Score(precision=0.8920863309352518, recall=0.2877030162412993, fmeasure=0.4350877192982456), mid=Score(precision=0.8920863309352518, recall=0.2877030162412993, fmeasure=0.4350877192982456), high=Score(precision=0.8920863309352518, recall=0.2877030162412993, fmeasure=0.4350877192982456)),
 'rougeLsum': AggregateScore(low=S

In [ ]:
output_string = StringIO()
with open("data/summary_two.pdf" , "rb") as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device =  TextConverter(rsrcmgr, output_string, laparams= LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
        #print('*' * 100)
        
print(output_string.getvalue())

Despite the fact that our planet is habitable only because most of it is
composed of water, it is the oceans that are the most immediately threatened
part of the earth.

It was in the oceans that life ﬁrst began to stir, shielded by the waters from the
sun’s irresistible radiation. It was from the oceans that planets and animals
emerged to colonize the land surface of the planet.  It is the oceans today that
provide the water vapour which, drawn up by the sun, falls upon the earth in
harvest-bringing, life-sustaining rain. The ocean is a major provider of the
oxygen released by its plankton for the beneﬁt of all the species of land, air and
sea – breathing with lungs and gills.

Without special qualities for holding heat, much of the earth would be
uninhabitable.

The oceans are the coolants of the tropics, the bringers of warm currents to
cold regions, the universal moderators of temperature throughout the globe.

The oceans are also indispensable to man because they ﬁrst created the


In [ ]:
text = output_string.getvalue()
text = text[:-4]
dataset = text.replace('\n', ' ').replace('\\', '').replace('\n\n', '').replace('\x0c','').replace('\x00', 'ff')
dataset

'Despite the fact that our planet is habitable only because most of it is composed of water, it is the oceans that are the most immediately threatened part of the earth.  It was in the oceans that life ﬁrst began to stir, shielded by the waters from the sun’s irresistible radiation. It was from the oceans that planets and animals emerged to colonize the land surface of the planet.  It is the oceans today that provide the water vapour which, drawn up by the sun, falls upon the earth in harvest-bringing, life-sustaining rain. The ocean is a major provider of the oxygen released by its plankton for the beneﬁt of all the species of land, air and sea – breathing with lungs and gills.  Without special qualities for holding heat, much of the earth would be uninhabitable.  The oceans are the coolants of the tropics, the bringers of warm currents to cold regions, the universal moderators of temperature throughout the globe.  The oceans are also indispensable to man because they ﬁrst created the

In [ ]:

n =512
bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)

metric.compute(predictions = [result], references = [dataset])

 the oceans provide the water vapour which, drawn up by the sun, falls upon the earth in harvest-bringing, life-bringing, harvest-bringing, life-sustaining ways. it is the oceans today that provide the water vapour which, drawn up by the sun, falls upon the earth in harvest-bringing, life-sustaining ways. the oceans are the coolants of the tropics, the bringers of warm currents to cold regions, the universal moderators of temperature throughout the globe. without special qualities for holding heat, much of the earth would be uninhabitable. in 1996, sixty-three million metric tons of fish came from the sea. fish could make up a large part of the protein diet required for the world’s children, especially those in developing countries, at a very low cost. rcent of the harvest from the oceans is converted to fish meal which today ends up feeding pigs and chickens in developed countries. it is very sad that ‘developed’ animal pets have the chance of a better diet than very many ‘developing’

{'rouge1': AggregateScore(low=Score(precision=0.9011627906976745, recall=0.4983922829581994, fmeasure=0.6418219461697724), mid=Score(precision=0.9011627906976745, recall=0.4983922829581994, fmeasure=0.6418219461697724), high=Score(precision=0.9011627906976745, recall=0.4983922829581994, fmeasure=0.6418219461697724)),
 'rouge2': AggregateScore(low=Score(precision=0.8011695906432749, recall=0.44193548387096776, fmeasure=0.5696465696465697), mid=Score(precision=0.8011695906432749, recall=0.44193548387096776, fmeasure=0.5696465696465697), high=Score(precision=0.8011695906432749, recall=0.44193548387096776, fmeasure=0.5696465696465697)),
 'rougeL': AggregateScore(low=Score(precision=0.7965116279069767, recall=0.4405144694533762, fmeasure=0.567287784679089), mid=Score(precision=0.7965116279069767, recall=0.4405144694533762, fmeasure=0.567287784679089), high=Score(precision=0.7965116279069767, recall=0.4405144694533762, fmeasure=0.567287784679089)),
 'rougeLsum': AggregateScore(low=Score(prec

In [ ]:

n =512
bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)

metric.compute(predictions = [result], references = [dataset])

 the oceans today provide the water vapour which falls upon the earth in harvest-bringing, life-bringing. it was in the oceans that life first began to stir, shielded by the waters from the sun’s irresistible radiation. the oceans are the coolants of the tropics, the bringers of warm currents to cold regions, the universal moderators of temperature throughout the globe. the oceans are also indispensable to man because they first created the worldwide currents of sebastian. in 1996, sixty-three million metric tons of fish came from the sea. fish could make up a large part of the protein diet required for the world’s children at a very low cost. rcent of the harvest from the oceans is converted to fish meal which ends up feeding pigs and chickens in developed countries. it is sad that ‘developed’ animal pets have the chance of a better diet than very many ‘developing’ babies.


{'rouge1': AggregateScore(low=Score(precision=0.9545454545454546, recall=0.47266881028938906, fmeasure=0.6322580645161291), mid=Score(precision=0.9545454545454546, recall=0.47266881028938906, fmeasure=0.6322580645161291), high=Score(precision=0.9545454545454546, recall=0.47266881028938906, fmeasure=0.6322580645161291)),
 'rouge2': AggregateScore(low=Score(precision=0.8562091503267973, recall=0.42258064516129035, fmeasure=0.5658747300215983), mid=Score(precision=0.8562091503267973, recall=0.42258064516129035, fmeasure=0.5658747300215983), high=Score(precision=0.8562091503267973, recall=0.42258064516129035, fmeasure=0.5658747300215983)),
 'rougeL': AggregateScore(low=Score(precision=0.8831168831168831, recall=0.43729903536977494, fmeasure=0.5849462365591398), mid=Score(precision=0.8831168831168831, recall=0.43729903536977494, fmeasure=0.5849462365591398), high=Score(precision=0.8831168831168831, recall=0.43729903536977494, fmeasure=0.5849462365591398)),
 'rougeLsum': AggregateScore(low=S

In [ ]:
output_string = StringIO()
with open("data/summary_three.pdf" , "rb") as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device =  TextConverter(rsrcmgr, output_string, laparams= LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
        #print('*' * 100)
        
print(output_string.getvalue())

It has been the custom of historians to divide the factors for wars into
immediate and underlying causes. Among these underlying causes, the
economic factor is generally placed at the head of the list. Indeed, the most
important of these was the industrial and commercial rivalry between
Germany and Great Britain.

Germany, after its uniﬁcation in 1871, went through a period of economic
miracle. By 1914, she was producing more iron and steel than Britain and
France combined. In chemicals, in dye, and in the manufacture of scientiﬁc
equipment she led the world. The products of her industries were crowding
British manufactures in nearly every market for continental Europe, in the Far
East and in Britain itself.

There is evidence that certain interests in Great Britain were becoming
seriously alarmed over the menace of German competition.

There seemed to be a strong conviction that Germany was waging deliberate
and deadly economic warfare upon Britain to capture her market by unfair
meth

In [ ]:
text = output_string.getvalue()
text = text[:-4]
dataset = text.replace('\n', ' ').replace('\\', '').replace('\n\n', '').replace('\x0c','').replace('\x00', 'ff')
dataset

'It has been the custom of historians to divide the factors for wars into immediate and underlying causes. Among these underlying causes, the economic factor is generally placed at the head of the list. Indeed, the most important of these was the industrial and commercial rivalry between Germany and Great Britain.  Germany, after its uniﬁcation in 1871, went through a period of economic miracle. By 1914, she was producing more iron and steel than Britain and France combined. In chemicals, in dye, and in the manufacture of scientiﬁc equipment she led the world. The products of her industries were crowding British manufactures in nearly every market for continental Europe, in the Far East and in Britain itself.  There is evidence that certain interests in Great Britain were becoming seriously alarmed over the menace of German competition.  There seemed to be a strong conviction that Germany was waging deliberate and deadly economic warfare upon Britain to capture her market by unfair met

In [ ]:

n =512
bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40,length_penalty=2.0, num_beams=4, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)

metric.compute(predictions = [result], references = [dataset])

 the industrial and commercial rivalry between Germany and Great Britain. by 1914, she was producing more iron and steel than Britain and France combined. by 1914, she was producing more iron and steel than Britain and France combined. david rothkopf: her industries were crowding British manufactures in nearly every market. he says there seemed to be a conviction that Germany was waging deadly economic warfare. rothkopf: for Britain to teeter on the edge of German competition, it would be a mistake. he says it would be a mistake for the u.s. to rely on british manufacturers. frida ghitis: o allow germany to be victorious in this struggle would mean the destruction of her prosperity. ghitis: there are indications that the french also were alarmed by the German industrial expansion. ghitis: o allow Germany to be victorious would mean the destruction of her national existence. ghitis: o allow Germany to be victorious in this struggle would mean the destruction of her prosperity. frida ghi

{'rouge1': AggregateScore(low=Score(precision=0.5954198473282443, recall=0.4588235294117647, fmeasure=0.5182724252491694), mid=Score(precision=0.5954198473282443, recall=0.4588235294117647, fmeasure=0.5182724252491694), high=Score(precision=0.5954198473282443, recall=0.4588235294117647, fmeasure=0.5182724252491694)),
 'rouge2': AggregateScore(low=Score(precision=0.44061302681992337, recall=0.3392330383480826, fmeasure=0.38333333333333336), mid=Score(precision=0.44061302681992337, recall=0.3392330383480826, fmeasure=0.38333333333333336), high=Score(precision=0.44061302681992337, recall=0.3392330383480826, fmeasure=0.38333333333333336)),
 'rougeL': AggregateScore(low=Score(precision=0.4961832061068702, recall=0.38235294117647056, fmeasure=0.4318936877076412), mid=Score(precision=0.4961832061068702, recall=0.38235294117647056, fmeasure=0.4318936877076412), high=Score(precision=0.4961832061068702, recall=0.38235294117647056, fmeasure=0.4318936877076412)),
 'rougeLsum': AggregateScore(low=S

In [ ]:

n =512
bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)

metric.compute(predictions = [result], references = [dataset])

 it has been the custom of historians to divide the factors for wars into immediate and underlying causes. the most important of these was the industrial and commercial rivalry between Germany and Great Britain. by 1914, she was producing more iron and steel than Britain and France combined. the products of her industries crowding British manufactures in nearly every market for continental Europe, in the far East and in Britain itself. there seems to be a strong conviction that Germany was waging deliberate and deadly economic warfare upon Britain to capture her market by unfair methods. in 1870, France had lost possession of the expensive iron and coal deposit of Lorraine. in 1870, the French had plenty of iron left in the Briery Fields. but they were afraid that their enemy might eventually reach ousted. france was under necessity of importing coal and this galled her pride almost as much as the loss of the iron. the Russian ambition to gain control of Constantinople conflicted with 

{'rouge1': AggregateScore(low=Score(precision=0.9267015706806283, recall=0.5205882352941177, fmeasure=0.6666666666666666), mid=Score(precision=0.9267015706806283, recall=0.5205882352941177, fmeasure=0.6666666666666666), high=Score(precision=0.9267015706806283, recall=0.5205882352941177, fmeasure=0.6666666666666666)),
 'rouge2': AggregateScore(low=Score(precision=0.8736842105263158, recall=0.4896755162241888, fmeasure=0.6275992438563327), mid=Score(precision=0.8736842105263158, recall=0.4896755162241888, fmeasure=0.6275992438563327), high=Score(precision=0.8736842105263158, recall=0.4896755162241888, fmeasure=0.6275992438563327)),
 'rougeL': AggregateScore(low=Score(precision=0.9214659685863874, recall=0.5176470588235295, fmeasure=0.6629001883239172), mid=Score(precision=0.9214659685863874, recall=0.5176470588235295, fmeasure=0.6629001883239172), high=Score(precision=0.9214659685863874, recall=0.5176470588235295, fmeasure=0.6629001883239172)),
 'rougeLsum': AggregateScore(low=Score(prec

In [ ]:
output_string = StringIO()
with open("data/summary_four.pdf" , "rb") as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device =  TextConverter(rsrcmgr, output_string, laparams= LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
        #print('*' * 100)
        
print(output_string.getvalue())

Mining ranks among the world’s most destructive industries. Yet mineral
extraction and processing are absent in most discussions of global
environmental threats. Governmental and private analyses have focused only
on increasing mineral supplies.

Each year, mining strips some 28 billion tons of material from the earth. This
is more than what is removed by the natural erosion of all the earth’s rivers.
Worldwide, mining and smelting generate an estimated 2.7 billion tons of
processing waste each year, much of it hazardous dwarﬁng the more familiar
municipal waste. Smelter pollution has created biological wastelands as large
as 10, 000 hectares and pumped some eight percent of the total worldwide
emissions of sulphur dioxide, a major contributor to acid rain, into the
atmosphere.

Mining could also cause more damaging deforestation than bad farming
practices in certain parts of the world. For example, smelters at a single iron
mine in Brazil will require enough fuelwood to deforest 50,00

In [ ]:
text = output_string.getvalue()
text = text[:-4]
dataset = text.replace('\n', ' ').replace('\\', '').replace('\n\n', '').replace('\x0c','').replace('\x00', 'ff')
dataset

'Mining ranks among the world’s most destructive industries. Yet mineral extraction and processing are absent in most discussions of global environmental threats. Governmental and private analyses have focused only on increasing mineral supplies.  Each year, mining strips some 28 billion tons of material from the earth. This is more than what is removed by the natural erosion of all the earth’s rivers. Worldwide, mining and smelting generate an estimated 2.7 billion tons of processing waste each year, much of it hazardous dwarﬁng the more familiar municipal waste. Smelter pollution has created biological wastelands as large as 10, 000 hectares and pumped some eight percent of the total worldwide emissions of sulphur dioxide, a major contributor to acid rain, into the atmosphere.  Mining could also cause more damaging deforestation than bad farming practices in certain parts of the world. For example, smelters at a single iron mine in Brazil will require enough fuelwood to deforest 50,0

In [ ]:

n =512
bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)

metric.compute(predictions = [result], references = [dataset])

 each year, mining strips some 28 billion tons of material from the earth. this is more than what is removed by the natural erosion of all the earth’s rivers. mining and smelting generate an estimated 2.7 billion tons of processing waste each year. smelter pollution has created biological wastelands as large as 10, 000 hectares. mining could cause more damaging deforestation than bad farming practices. mining could cause more damaging deforestation than bad farming practices. mining has been poorly regulated even in wealthy industrialized nations. many governments subsidize mineral production, but few enact or enforce strict environmental regulations. prices of minerals do not include their full environmental cost. osts of eroded land, dammed or polluted rivers and displacement of people unlucky enough to live atop mineral deposits. the devastating effects of the industry are particularly severe in the developing countries. the people of most mineral–exporting countries gain little fro

{'rouge1': AggregateScore(low=Score(precision=0.9488372093023256, recall=0.4523281596452328, fmeasure=0.6126126126126126), mid=Score(precision=0.9488372093023256, recall=0.4523281596452328, fmeasure=0.6126126126126126), high=Score(precision=0.9488372093023256, recall=0.4523281596452328, fmeasure=0.6126126126126126)),
 'rouge2': AggregateScore(low=Score(precision=0.8644859813084113, recall=0.4111111111111111, fmeasure=0.5572289156626505), mid=Score(precision=0.8644859813084113, recall=0.4111111111111111, fmeasure=0.5572289156626505), high=Score(precision=0.8644859813084113, recall=0.4111111111111111, fmeasure=0.5572289156626505)),
 'rougeL': AggregateScore(low=Score(precision=0.9302325581395349, recall=0.4434589800443459, fmeasure=0.6006006006006006), mid=Score(precision=0.9302325581395349, recall=0.4434589800443459, fmeasure=0.6006006006006006), high=Score(precision=0.9302325581395349, recall=0.4434589800443459, fmeasure=0.6006006006006006)),
 'rougeLsum': AggregateScore(low=Score(prec

In [ ]:

n =512
bank = [dataset[i:i+n] for i in range(0, len(dataset), n)]

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")


summary = []
for data in bank:
    #inputs, references = batch
    inputs = tokenizer("summarize: " + data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True, no_repeat_ngram_size=2)
    summary.append(tokenizer.decode(outputs[0]))
    


    #print(tokenizer.decode(outputs[0]))
    
result  = "".join(summary)
result = result.replace(r'<pad>','').replace(r'</s>','')

print(result)

metric.compute(predictions = [result], references = [dataset])

 each year, mining strips 28 billion tons of material from the earth. this is more than what is removed by the natural erosion of all the earth’s rivers. worldwide, mining and smelting generate an estimated 2.7 billion tons of processing waste each year. smelter pollution has created biological wastelands as large as 10, 000 hectares. mining could also cause more damaging deforestation than bad farming practices. mining could also cause more damaging deforestation than bad farming practices in certain parts of the world. mining has been poorly regulated even in wealthy industrialized nations. some governments enact or enforce strict environmental regulations for mining operations. prices of minerals do not include their full environmental cost. osts of eroded land, dammed or polluted rivers and displacement of people unlucky enough to live atop mineral deposits. governments should remove subsidies provided for mining virgin minerals. the people of most mineral–exporting countries gain 

{'rouge1': AggregateScore(low=Score(precision=0.9301310043668122, recall=0.4722838137472284, fmeasure=0.6264705882352941), mid=Score(precision=0.9301310043668122, recall=0.4722838137472284, fmeasure=0.6264705882352941), high=Score(precision=0.9301310043668122, recall=0.4722838137472284, fmeasure=0.6264705882352941)),
 'rouge2': AggregateScore(low=Score(precision=0.8245614035087719, recall=0.4177777777777778, fmeasure=0.5545722713864307), mid=Score(precision=0.8245614035087719, recall=0.4177777777777778, fmeasure=0.5545722713864307), high=Score(precision=0.8245614035087719, recall=0.4177777777777778, fmeasure=0.5545722713864307)),
 'rougeL': AggregateScore(low=Score(precision=0.8777292576419214, recall=0.44567627494456763, fmeasure=0.5911764705882353), mid=Score(precision=0.8777292576419214, recall=0.44567627494456763, fmeasure=0.5911764705882353), high=Score(precision=0.8777292576419214, recall=0.44567627494456763, fmeasure=0.5911764705882353)),
 'rougeLsum': AggregateScore(low=Score(p

Reference
1. What is Rouge score? [here](https://www.youtube.com/watch?v=TMshhnrEXlg)
2. Interpreting Rouge scores [here](https://stats.stackexchange.com/questions/301626/interpreting-rouge-scores)
3. How others tested their text summarizer after using rouge [here](https://link.springer.com/article/10.1007/s10579-017-9389-4)
4. Understanding how to finetune Hyperparameters for model.generate [here](https://huggingface.co/blog/how-to-generate)
5. Understanding num_beam used as an argument for model.generate [here](https://huggingface.co/blog/how-to-generate)